## This notebook is create sample site along the street map
1. Prepare the street map
2. Set the sampling distance along the street


### 1. Clean the street map remove some types of streets
If you download the street map from open street map, you may need to remove some types of streets for your analyses, like highways, motor ways.

In [1]:
import fiona
import os,os.path
from shapely.geometry import shape,mapping
from shapely.ops import transform
from functools import partial
import pyproj
from fiona.crs import from_epsg
from shapely.geometry import LineString, Point # To create line geometries that can be used in a GeoDataFrame
import math


count = 0
# s = {'trunk_link','tertiary','motorway','motorway_link','steps', None, ' ','pedestrian','primary', 'primary_link','footway','tertiary_link', 'trunk','secondary','secondary_link','tertiary_link','bridleway','service'}
# s = {'trunk_link','tertiary','motorway','motorway_link','steps', ' ','pedestrian','primary', 'primary_link','footway','tertiary_link', 'trunk','secondary','secondary_link','tertiary_link','bridleway','service'}
s = {}


# set as your own filename
inshp = '../sample-spatialdata/CambridgeStreet_wgs84.shp'
outshp = '../sample-spatialdata/Cambridge20m.shp'


inshp = '/Users/xiaojiang/Dropbox (MIT)/ResearchProj/treepedia/cities-proj/chihuahua/red_vial_wgs.shp'
outshp = '/Users/xiaojiang/Dropbox (MIT)/ResearchProj/treepedia/cities-proj/chihuahua/street20m.shp'
# specify the distance
mini_dist = 20


# the temporaray file of the cleaned data
root = os.path.dirname(inshp)
basename = 'clean_' + os.path.basename(inshp)
temp_cleanedStreetmap = os.path.join(root,basename)

# if the tempfile exist then delete it
if os.path.exists(temp_cleanedStreetmap):
    fiona.remove(temp_cleanedStreetmap, 'ESRI Shapefile')
    print ('removed the existed tempfile')
    

removed the existed tempfile


In [2]:
# clean the original street maps by removing highways, if it the street map not from Open street data, users'd better to clean the data themselve
with fiona.open(inshp) as source, fiona.open(temp_cleanedStreetmap, 'w', driver=source.driver, crs=source.crs,schema=source.schema) as dest:
    for feat in source:
        try:
            i = feat['properties']['highway'] # for the OSM street data
            # i = feat['properties']['fclass'] # for the OSM tokyo street data
            if i in s:
                continue
        except:
            # if the street map is not osm, do nothing. You'd better to clean the street map, if you don't want to map the GVI for highways
            key = list(dest.schema['properties'].keys())[0]
            # key = dest.schema['properties'].keys()[0] # get the field of the input shapefile and duplicate the input feature
            i = feat['properties'][key]
            if i in s:
                continue
        
        # print feat
        dest.write(feat)
    
schema = {
    'geometry': 'Point',
    'properties': {'id': 'int'},
}


### 2. Example of using Pyproj to do projection transform

In [36]:
import numpy as np
import pyproj

# Use the "new" numpy API for the random number generation (requires numpy >=1.17)
rng = np.random.default_rng(seed=1)

# Create arrays with random points
no_points = 5
longitudes = rng.uniform(low=-110, high=-100, size=no_points)
latitudes = rng.uniform(low=33, high=45, size=no_points)
print("  Longitude \t Latitude")
print(np.column_stack((longitudes, latitudes)))
print()

# Create the transformer object. 
# You can create the object using either integers or "epsg" strings 
transformer = pyproj.Transformer.from_crs(crs_from=4326, crs_to=3857)
#             pyproj.Transformer.from_crs(crs_from="epsg:4326", crs_to="epsg:3857")

# Apply the transformation. 
# Be careful! The order of the output here is Y/X
y1, x1 = transformer.transform(longitudes, latitudes)
# print(np.column_stack((x1, y1)))
# print()

# If you prefer to work with X/Y instead you need to set `always_xy=True` to the transformer object
transformer = pyproj.Transformer.from_crs(crs_from=4326, crs_to=3857, always_xy=True)
x2, y2 = transformer.transform(longitudes, latitudes)
print(np.column_stack((x2, y2)))
print()

  Longitude 	 Latitude
[[-104.88178375   38.07991739]
 [-100.49536304   42.93243113]
 [-108.55840387   37.91038964]
 [-100.51350553   39.59512425]
 [-106.88168548   33.33070936]]

[[-11675386.76087391   4590721.61904675]
 [-11187092.64033514   5301692.82389603]
 [-12084666.24045104   4566774.59659932]
 [-11189112.25329371   4807280.09689674]
 [-11898014.80274875   3939282.81983831]]



### Create sample sites along the streets
Be careful of the pyproj version for the transform

In [30]:
from pyproj import Transformer
import math

# the transformer used to switch between the projections unit in degree to meter
transformer = Transformer.from_crs(4326, 3857, always_xy=True) #don't miss always_xy=True
transformer_back = Transformer.from_crs(3857, 4326, always_xy=True)


# Create point along the streets
with fiona.Env():
    #with fiona.open(outshp, 'w', 'ESRI Shapefile', crs=source.crs, schema) as output:
    with fiona.open(outshp, 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = schema) as output:
        i = 0
        for line in fiona.open(temp_cleanedStreetmap):
            i = i + 1
            if i %1000 == 0: print(i)
            
            # try: 
            # deal with MultiLineString and LineString
            featureType = line['geometry']['type']
            
            # for the LineString
            if featureType == "LineString":
                first = shape(line['geometry'])
                length = first.length
                
                # deal with different version of pyproj
                if pyproj.__version__[0] != '2':
                    project = partial(pyproj.transform,pyproj.Proj(init='EPSG:4326'),pyproj.Proj(init='EPSG:3857')) #3857 is psudo WGS84 the unit is meter
                    line2 = transform(project, first)
                    print('using partial')
                else:
                    # loop all vertices in the line and then reproj
                    line2_coord = []
                    for (lon, lat) in first.coords:    
                        x, y = transformer.transform(lon, lat)
                        line2_coord.append((x, y))
                        
                    line2 = LineString(line2_coord)
                
                linestr = list(line2.coords)
                dist = mini_dist 
                
                if math.isnan(line2.length): continue
                
                # create sample points along lines and save to shapefile
                for distance in range(0,int(line2.length), dist):
                    point = line2.interpolate(distance)
                    
                    if pyproj.__version__[0]!='2':
                        project2 = partial(pyproj.transform,pyproj.Proj(init='EPSG:3857'),pyproj.Proj(init='EPSG:4326')) #3857 is psudo WGS84 the unit is meter
                        point = transform(project2, point)
                    else:
                        point = Point(transformer_back.transform(point.x, point.y))
                    output.write({'geometry':mapping(point),'properties': {'id':1}})
            
            # for the MultiLineString, seperate these lines, then partition those lines
            elif featureType == "MultiLineString":
                multiline_geom = shape(line['geometry'])
                print ('This is a multiline')
                for singleLine in multiline_geom:
                    length = singleLine.length
                    
                    if pyproj.__version__[0]!='2':
                        project = partial(pyproj.transform,pyproj.Proj(init='EPSG:4326'),pyproj.Proj(init='EPSG:3857')) #3857 is psudo WGS84 the unit is meter
                        line2 = transform(project, singleLine)
                    else:
                        # loop all vertices in the line and then reproj
                        line2_coord = []
                        for (lon, lat) in singleLine.coords:    
                            x, y = transformer.transform(lon, lat)
                            line2_coord.append((x, y))
                            
                        line2 = LineString(line2_coord)
                    
                    linestr = list(line2.coords)
                    dist = mini_dist #set
                    
                    if math.isnan(line2.length): continue
                    
                    for distance in range(0,int(line2.length), dist):
                        point = line2.interpolate(distance)
                        
                        if pyproj.__version__[0]!='2':
                            project2 = partial(pyproj.transform,pyproj.Proj(init='EPSG:3857'),pyproj.Proj(init='EPSG:4326')) #3857 is psudo WGS84 the unit is meter
                            point = transform(project2, point)
                        else:
                            point = Point(transformer_back.transform(point.x, point.y))
                        
                        output.write({'geometry':mapping(point),'properties': {'id':1}})
                        
                else:
                    print('Else--------')
                    continue
                
            # except:
            #     print ("You should make sure the input shapefile is WGS84")
            #     return

print("Process Complete")

# delete the temprary cleaned shapefile
# fiona.remove(temp_cleanedStreetmap, 'ESRI Shapefile')


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
This is a multiline
Else--------
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
1540